In [ ]:
from robot_properties_solo.solo12wrapper import Solo12Config
from robot_properties_solo.solo12wrapper import Solo12Robot
from bullet_utils.env import BulletEnvWithGround
from simulate_solo import Simulator
import conf_solo12_trot as conf

# initialize robot in pybullet
env = BulletEnvWithGround()
simulator = Simulator(env, Solo12Robot(), conf)
robot = Solo12Config.buildRobotWrapper()
q0, dq0 = simulator.q0, simulator.nu0
simulator.robot.reset_state(q0, dq0) 
simulator.robot.pin_robot.framesForwardKinematics(q0)

In [ ]:
# display whole-body motion in meshcat viewer
from wholebody_croccodyl_solver import WholeBodyDDPSolver
from wholebody_croccodyl_model import WholeBodyModel
import pinocchio as pin
import sys

wbd_model = WholeBodyModel(conf)
ddp_solver = WholeBodyDDPSolver(wbd_model, MPC=True, WARM_START=True)

sol = ddp_solver.run_OL_MPC()

if conf.WITH_MESHCAT_DISPLAY:
    viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model)
    try:
        viz.initViewer(open=True)
    except ImportError as err:
        print(
            "Error while initializing the viewer. Make sure you installed Python meshcat"
        )
        print(err)
        sys.exit(0)
    viz.loadViewerModel()
    for sol_k in sol:
        q = sol_k['jointPos'][0]        
        viz.display(q)

In [ ]:
# simulate motion in pybullet
import numpy as np
wbd_model = WholeBodyModel(conf)
ddp_solver = WholeBodyDDPSolver(wbd_model, MPC=True, WARM_START=True)
running_models = ddp_solver.whole_body_model.running_models
N_traj, N_mpc = ddp_solver.N_traj, ddp_solver.N_mpc
Kp = 2*np.eye(12)
Kd = 0.01*np.eye(12)
N_inner = int(ddp_solver.dt/ddp_solver.dt_ctrl)
for traj_time_idx in range(N_traj):
    # update models
    current_running_models = running_models[traj_time_idx:traj_time_idx+N_mpc]
    current_terminal_model = current_running_models[-1]
    ddp_solver.update_ocp(current_running_models, current_terminal_model)
    if traj_time_idx == 0:
        if ddp_solver.WARM_START:
            ddp_solver.solver.solve(ddp_solver.x_init, ddp_solver.u_init)
        else:
            ddp_solver.solve()
    else:
        ddp_solver.solve(xs, us)
    sol_k = ddp_solver.get_solution_trajectories()
    q_des, qdot_des = sol_k['jointPos'][0][7:], sol_k['jointVel'][0][6:]
    tau_ff = sol_k['jointTorques'][0]
    for _ in range(N_inner):
        q_k, dq_k = simulator.robot.get_state()
        tau_k = tau_ff + \
                  Kp @ (q_des - q_k[7:]) + \
                  Kd @ (qdot_des - dq_k[6:])
        simulator.robot.send_joint_command(tau_k)
        # step simulation 
        simulator.env.step(sleep=True)     
    q_k, dq_k = simulator.robot.get_state()
    # warm-start ocp from the previous solution
    x = [ddp_solver.solver.xs[i] for i in range(len(ddp_solver.solver.xs))]
    u = [ddp_solver.solver.us[i] for i in range(len(ddp_solver.solver.us))]
    us = u[1:] + [u[-1]]    
    xs = x[1:] + [x[-1]]
    # OL-MPC (initial state = predicted state)
    x0 = xs[0]
    # CL-MPC (initial state = measured state)
    # x0 = np.concatenate([q_k, dq_k])
    ddp_solver.x0 = x0
